In [1]:
%%time
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#hyperparameters. These can be changed to reduce the loss.
#48,96,64
block_size = 64 #sequence length
batch_size = 128 #how many of the batches we want to happen at the same time
#common learning rates to use are 1e-3, 1e-4, 3e-3, 3e-4
learning_rate = 3e-4 #learning rate can be experimented on to evaluate which value produces the best prformance and qaulity over time.
max_iters = 20000 #iterations we want to do
eval_iters = 100 #reports loss
n_embd = 384 #the total dimensions we want to capture from all heads concatonated together.
n_layer = 8 #number of decoder blocks you want to have
n_head = 8 #number of heads we have running in parallel
dropout = 0.2

cpu
CPU times: user 1.93 s, sys: 377 ms, total: 2.31 s
Wall time: 2 s


In [2]:
chars = ""
with open('plaintxt1.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)

In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data)) #the 0.8 represents 80% of the data
training_data = data[:n]
validation_data = data[n:]
#The chunk of code above is for producing the training set and validation set split

def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix) #This causes the long prints in the optimiser cell
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) #the .to(device) forces the batch onto the gpu or specified device
    return x,y

In [5]:
@torch.no_grad() #this is a decorator. It makes sure that Pytorch doesnt use gradient here. this improves performance, memory usage, computation etc...
def estimate_loss():
    out = {}
    model.eval() # this puts the model into evaliuation mode which allows it to be validated/evaluated at its optimal form and test it.
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() #this puts the model into training mode allowing it to update the weights anf biases.
    return out

In [6]:
#building Scaled Dot Procuct attention 
class Head(nn.Module):
    """single head of self-attention"""
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) #registers the no look ahead masking to the models state so it starts in a masked state
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #input of size (Batch, Time-step, Channels)
        #output of size (Batch, Time-step, Head Size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs) shape
        q = self.query(x) # (B,T,hs) shape
        #compute the attention scores for the characters
        weights = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 #Dot products the keys and queries. The transposing allows for the grid multiplication. Flips second last dimensin (-2) with last dimension (-1). The scaling part makes sure no dot product is too overpowering allowing us to hear all parts of the input equally
        weights = weights.masked_fill(self.tril[:T,:T] == 0, float('-inf')) #masking to make sure the model cant look ahead at future parameters and cheat. Each time step exposes the next value in the tensor.
        weights = F.softmax(weights, dim=-1) #apply softmax function to get (B, T, T). The softmax makes the model more confident in it's predictions
        weights = self.dropout(weights)
        #performs weighted aggregation of the value
        v = self.value(x) # (B,T,hs)
        out = weights @ v #multiply value weight to softmax value. (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

#building MultiHeadAttention connection within decoder
class MultiHeadAttention(nn.Module):
    """multible heads of self attention in parallel"""
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) #self.heads is a module list. He havehaeds in parallel for each head
        self.proj = nn.Linear(head_size * num_heads, n_embd) #this line project the headds_size miltiplied by the number of heads to an n_embd. Adds more learnable parameters to help model learn more about the given text
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #concatonating each head together along the last dimension (channel dimension) (B,T,C) -> (B,T,[h1,h1,h1,h1,h2,h2,h2,h2,h3,h3...])
        out = self.dropout(self.proj(out))
        return out

#building feedforward connection within decoder block
class FeedForward(nn.Module):
    """sample linear layer followed by a non-linear layer (in this case ReLU activation) and then another linear layer"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout), #makes percentage of neurons drop out and become zero
        )
    def forward(self, x):
        return self.net(x)

#building the decoder blocks
class Block(nn.Module):
    """Transformer block: communication followed by computation"""
    def __init__(self, n_embd, n_head):
        #svsvsvsvsvsv
        super().__init__()
        head_size = n_embd // n_head #n_embed is the number of features we have and n_head is the number of heads we have
        self.sa = MultiHeadAttention(n_head, head_size) #sa is short for self-attention
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    #post-norm or pre-norm architecture. We use post-norm here because it converges better for the data and model parameters being used.
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

#Building language model
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) #creating an embedding table. This is basically a look-up table. This is a grid with tokens where the probability for a predicted token can be seen.
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # n_embd will be used in positional and token embedding to store tokens in very last vectors.
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) #no of decoder blocks we run sequentially. The asterix repeats the block part for the number of n_layers
        self.ln_f = nn.LayerNorm(n_embd) # final layer normalisation
        self.lm_head = nn.Linear(n_embd, vocab_size) # compressing transformation for making it softmax workable.
        
        #this next chunk of code initialises the weights around a standard deviation of 0.02 so that the initial weights are somewhat consostant
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    #the forward pass function descripes how inputs into the network will be passed trough the layers.
    #.view turns a matrix unpacked into x, y, z, etc... coordinates back into a tensor.
    def forward(self, index, targets=None):
        B, T = index.shape

        # idx and targets are both (B,T) tensor of integers
        token_embed = self.token_embedding_table(index)
        position_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = token_embed + position_embed # (B,T,C) shape
        x = self.blocks(x) # feeding in x parameters in (B,T,C) shape
        x = self.ln_f(x) # once the parameters have gone through all layers, they are fed into the final layer norma in (B,T,C) shape
        logits = self.lm_head(x) # use this line to get the parametes into probabilitis which van be fed into the softmax function in (B,T,vocabulary_size) shape
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #B is for batch, T is for time, Channels is vocabulary size
            logits = logits.view(B*T, C) #because we're paying attention to the vocabulary, or the Channles we can blent the batch and time. As long a logits and targets have the same batch and time then this will be fine.
            #B and T are multiplied because Pytorch expects an input shape of B by C by etc... (e.g B, C, T) but the shape is B by T by C (B, T, C), so the B and T are combined into one.
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) #cross_entropy is a loss function

        return logits, loss #logits are basically a bunch of floating point numbers which are normalised. They show the contribution of a single token to the whole embedding, basically a probability distribution for what you want to predict

    def generate(self, index, max_new_tokens): #max_new_tokens indicates the max length of the generated text/tokens
        # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #getting the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            #applying softmax funtion to get probability distribution.
            probs = F.softmax(logits, dim=-1) #(B, C)
            #sample from the probability distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #add the samples index to the running sequence of indices
            index = torch.cat((index,index_next), dim=1) #(B, T+1)
           
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device) #runs the model on specified device

#context = torch.zeros((1,1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

In [7]:
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate) #Adam optimisation algorithm used for optimiser.

#Getting an idea of hor the model is performing over time
for iter in range(max_iters): #iters refers to the iterations, or no of training iterations
    if iter % eval_iters == 0:
        #reporting losses as the model trains
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}") #Prints the iteration step and loss for itaerations divisible by eval_iters value
    
    #sampling batch of data xb is x batch and yb is y batch
    xb, yb = get_batch('train')
    
    #evaluate the loss using loss function
    logits, loss = model.forward(xb, yb)
    #optimising only with the current gradient of the current data
    optimiser.zero_grad(set_to_none=True) #Pytorch accumulates and adds the gradients over time by default. Using .zero_grad makes stops the accumilation of preveous gradients from happening. This allows for previous sketchy gradients and data to not be influence current ones.
    loss.backward() #This is a backward bass. Basically a reverse pass through
    optimiser.step()
print(loss.item())

KeyboardInterrupt: 

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)